
##**Open Weather Map API**

In [ ]:
import pandas as pd
!pip install PyMySQL
import pymysql
import sqlalchemy 
# sample cities DataFrame for exercises

cities = pd.DataFrame(data = {'City_id' : [1],'City' : ['Berlin'], 'Country_code' : ['DE']})
cities

**API key**

In [ ]:
# please use your own API key
OWM_key = 'cf36567c4cb288fd1cd39a18cbfc1b5b'

## Approach without a wrapper

We are going to use 5 day / 3 hour forecast https://openweathermap.org/forecast5

Provided at their docmentation you can find an example of API call with only having info about city name and country_code https://openweathermap.org/forecast5#name5

api.openweathermap.org/data/2.5/forecast?q={city name},{country code}&appid={API key}

city name and country code have to be in format e.g. ('Berin,DE')
appid is your unique API key
you can also add units which can be 'standard', 'metric', 'imperial'
or lang for languages e.g. 'de', 'it' etc.
or cnt for number of timestamps in response
You can also find an example of API call with having info about latitude and longitude
https://openweathermap.org/forecast5#geo5

api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

You can choose your own approach. We will show here the approach with city name and country code

In [ ]:
import requests
url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['city'][0]},{cities['country_code'][0]}&appid={OWM_key}&units=metric"
response = requests.get(url)
response.status_code
#response.json()

In [ ]:
weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
# iterating over items in 'list' node and selecting the subnodes gives us the exact info we need 
for hour in response.json()['list']:
  weather_dict['city_id'].append(cities['city_id'][0])
  weather_dict['temperature'].append(hour['main']['temp'])
  weather_dict['wind_speed'].append(hour['wind']['speed'])
  weather_dict['forecast_time'].append(hour['dt_txt'])
  weather_dict['outlook'].append(hour['weather'][0]['description'])
  weather_dict['temperature_feels_like'].append(hour['main']['feels_like'])
  weather_dict['pop'].append(hour['pop'])

 **Creating a new weather DataFrame**

In [ ]:
weather_df = pd.DataFrame(weather_dict)

In [ ]:
schema="gans_database"   
host="127.0.0.1"        
user="root"
password="my_module.mypassword" 
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}' 

In [ ]:
weather_df.to_sql('weathers',        
              if_exists='append', 
              con=con,            
              index=False)